# Seasonal Autoregressive Integrated Moving Averages Model with Exogenous Variables

In [1]:
!pip install pmdarima

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pmdarima as pm
import statsmodels.api as sm

In [2]:
daily_data = pd.DataFrame(pd.read_csv("consumption_daily.csv",header=0, index_col=0, parse_dates=True, squeeze=True))
daily_data_train= pd.DataFrame(daily_data["2016":"2018"],copy=True)
daily_data_test = pd.DataFrame(daily_data["2019"],copy=True)

In [3]:
tatil_data = pd.DataFrame(np.zeros(shape=(daily_data.count(0)[0],3)),index=daily_data.index,columns=["Tatil","Bayram","Haftasonu"])
tatil_gunleri = [str(x)+ a for a in ["-01-01","-04-23","-05-01","-05-19","-07-15","-08-30","-10-29"] for x in range(2016,2020)]
tatil_data['Haftasonu'][(daily_data.index.dayofweek==5)|(daily_data.index.dayofweek==6)] = 1
for gun in tatil_gunleri:
    tatil_data["Tatil"].loc[gun] = 1

bayramlar = ['2016-07-04', '2016-07-05', '2016-07-06', '2016-07-07',
             '2016-09-11', '2016-09-12', '2016-09-13', '2016-09-14','2016-09-15',
             '2017-06-24', '2017-06-25', '2017-06-26', '2017-06-27',
             '2017-08-31', '2017-09-01', '2017-09-02', '2017-09-03', '2017-09-04',
             '2018-06-14', '2018-06-15', '2018-06-16', '2018-06-17',
             '2018-08-20', '2018-08-21', '2018-08-22', '2018-08-23', '2018-08-24', 
             '2019-06-04', '2019-06-05', '2019-06-06', '2019-06-07', 
             '2019-08-10', '2019-08-11', '2019-08-12', '2019-08-13', '2019-08-14']
#Setting the corresponding dates to 1.
for gun in tatil_gunleri:
    tatil_data["Tatil"].loc[gun] = 1
for gun in bayramlar:
    tatil_data["Bayram"].loc[gun] =1

In [4]:
tatil_data.head()

,Tatil,Bayram,Haftasonu
Date,,,
2016-01-01,1.0,0.0,0.0
2016-01-02,0.0,0.0,1.0
2016-01-03,0.0,0.0,1.0
2016-01-04,0.0,0.0,0.0
2016-01-05,0.0,0.0,0.0


## Searching Models Stepwise through SARIMAX(p,d,q)(P,D,Q)(7) parameters
## Building Models with different exogenous features

In [5]:
model_noExog = pm.auto_arima(daily_data[:"2018"],
                     max_d = 2, max_p=10, max_q=10, max_P=10, max_Q=10, max_D = 10, seasonal=True,
                     m=7,
                     stepwise=True, suppress_warnings=True, with_intercept=True,
                     error_action='ignore',maxiter = 50)



In [6]:
bayram_df = pd.DataFrame(tatil_data[:"2018"]["Bayram"])
bayram_df.index = daily_data[:"2018"].index

In [7]:
model_onlyBayram = pm.auto_arima(daily_data["Consumption"][:"2018"], exogenous=bayram_df,
                     max_d = 2, max_p=10, max_q=10, max_P=10, max_Q=10, max_D = 10, seasonal=True,
                     m=7,
                     stepwise=True, suppress_warnings=True, with_intercept=True,
                     error_action='ignore',maxiter = 50)

In [8]:
tatil_df = pd.DataFrame(tatil_data[:"2018"]["Tatil"])
tatil_df.index = daily_data[:"2018"].index

In [9]:
model_onlyTatil = pm.auto_arima(daily_data["Consumption"][:"2018"], exogenous=tatil_df,
                     max_d = 2, max_p=10, max_q=10, max_P=10, max_Q=10, max_D = 10, seasonal=True,
                     m=7,
                     stepwise=True, suppress_warnings=True, with_intercept=True,
                     error_action='ignore',maxiter = 50)

In [10]:
haftasonu_df = pd.DataFrame(tatil_data[:"2018"]["Haftasonu"])
haftasonu_df.index = daily_data[:"2018"].index

In [11]:
model_onlyHaftasonu = pm.auto_arima(daily_data["Consumption"][:"2018"], exogenous=haftasonu_df,
                     max_d = 2, max_p=10, max_q=10, max_P=10, max_Q=10, max_D = 10, seasonal=True,
                     m=7,
                     stepwise=True, suppress_warnings=True, with_intercept=True,
                     error_action='ignore',maxiter = 50)

In [12]:
bayram_tatil_df = pd.DataFrame(tatil_data[:"2018"][["Bayram","Tatil"]])
bayram_tatil_df.index = daily_data[:"2018"].index


In [13]:
model_noHaftasonu = pm.auto_arima(daily_data["Consumption"][:"2018"], exogenous=bayram_tatil_df,
                     max_d = 2, max_p=10, max_q=10, max_P=10, max_Q=10, max_D = 10, seasonal=True,
                     m=7,
                     stepwise=True, suppress_warnings=True, with_intercept=True,
                     error_action='ignore',maxiter = 50)

In [14]:
haftasonu_tatil_df = pd.DataFrame(tatil_data[:"2018"][["Tatil","Haftasonu"]])
haftasonu_tatil_df.index = daily_data[:"2018"].index


In [15]:
model_noBayram = pm.auto_arima(daily_data["Consumption"][:"2018"], exogenous=haftasonu_tatil_df,
                     max_d = 2, max_p=10, max_q=10, max_P=10, max_Q=10, max_D = 10, seasonal=True,
                     m=7,
                     stepwise=True, suppress_warnings=True, with_intercept=True,
                     error_action='ignore',maxiter = 50)

In [16]:
bayram_hs_df = pd.DataFrame(tatil_data[:"2018"][["Bayram","Haftasonu"]])
bayram_hs_df.index = daily_data[:"2018"].index


In [17]:
model_noTatil = pm.auto_arima(daily_data["Consumption"][:"2018"], exogenous=bayram_hs_df,
                     max_d = 2, max_p=10, max_q=10, max_P=10, max_Q=10, max_D = 10, seasonal=True,
                     m=7,
                     stepwise=True, suppress_warnings=True, with_intercept=True,
                     error_action='ignore',maxiter = 50)

In [18]:
model_all = pm.auto_arima(daily_data["Consumption"][:"2018"], exogenous=tatil_data[:"2018"],
                     max_d = 2, max_p=10, max_q=10, max_P=10, max_Q=10, max_D = 10, seasonal=True,
                     m=7,
                     stepwise=True, suppress_warnings=True, with_intercept=True,
                     error_action='ignore',maxiter = 50)


In [19]:
#Models Dictionary
models = {"No Exogenous Model":model_noExog,
          "Only Bayram Exog. Model":model_onlyBayram,
          "Only Tatil Exog. Model":model_onlyTatil,
          "Only Haftasonu Exog. Model":model_onlyHaftasonu,
          "No Bayram Model":model_noBayram,
          "No Haftasonu Model":model_noHaftasonu,
          "No Tatil Model":model_noTatil,
          "All Including Model":model_all}

In [20]:

model_all.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                      
===========================================================================================
Dep. Variable:                                   y   No. Observations:                 1096
Model:             SARIMAX(3, 1, 2)x(1, 0, [1], 7)   Log Likelihood              -12834.515
Date:                             Wed, 01 Jul 2020   AIC                          25693.030
Time:                                     01:39:26   BIC                          25753.012
Sample:                                 01-01-2016   HQIC                         25715.727
                                      - 12-31-2018                                         
Covariance Type:                               opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      3.1079      6.092      0.510      0.610      -8.833      15.049
Tatil       8901.1674   5980.198      1.488      0.137   -2819.804    2.06e+04
Bayram     -7.312e+04   9488.346     -7.707      0.000   -9.17e+04   -5.45e+04
Haftasonu  -5.077e+04   1.19e+04     -4.283      0.000    -7.4e+04   -2.75e+04
ar.L1          0.0112      0.733      0.015      0.988      -1.425       1.448
ar.L2          0.9377      0.738      1.270      0.204      -0.509       2.385
ar.L3         -0.0734      0.057     -1.278      0.201      -0.186       0.039
ma.L1          0.0086      0.773      0.011      0.991      -1.506       1.523
ma.L2         -0.9914      0.766     -1.294      0.196      -2.493       0.510
ar.S.L7        0.9620      0.009    107.035      0.000       0.944       0.980
ma.S.L7       -0.8005      0.023    -35.569      0.000      -0.845      -0.756
sigma2       1.48e+09      0.018   8.42e+10      0.000    1.48e+09    1.48e+09
===================================================================================
Ljung-Box (Q):                      350.09   Jarque-Bera (JB):              3303.96
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               0.60   Skew:                            -0.12
Prob(H) (two-sided):                  0.00   Kurtosis:                        11.51
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
[2] Covariance matrix is singular or near-singular, with condition number 6.64e+27. Standard errors may be unstable.
"""

### Fitting models

In [21]:
model_noExog_fit = model_noExog.fit(daily_data["Consumption"][:"2018"])

model_onlyBayram_fit = model_onlyBayram.fit(daily_data["Consumption"][:"2018"],exogenous=bayram_df)
model_onlyTatil_fit = model_onlyBayram.fit(daily_data["Consumption"][:"2018"],exogenous=tatil_df)
model_onlyHaftasonu_fit = model_onlyBayram.fit(daily_data["Consumption"][:"2018"],exogenous=haftasonu_df)

model_noHaftasonu_fit = model_noHaftasonu.fit(daily_data["Consumption"][:"2018"],exogenous=bayram_tatil_df)
model_noTatil_fit = model_noTatil.fit(daily_data["Consumption"][:"2018"],exogenous=bayram_hs_df)
model_noBayram_fit = model_noTatil.fit(daily_data["Consumption"][:"2018"],exogenous=haftasonu_tatil_df)

model_all_fit = model_all.fit(daily_data["Consumption"][:"2018"],exogenous=tatil_data[:"2018"])


models_fit = {"No Exogenous Model":model_noExog_fit,
          "Only Bayram Exog. Model":model_onlyBayram_fit,
          "Only Tatil Exog. Model":model_onlyTatil_fit,
          "Only Haftasonu Exog. Model":model_onlyHaftasonu_fit,
          "No Bayram Exog. Model":model_noBayram_fit,
          "No Haftasonu Model":model_noHaftasonu_fit,
          "No Tatil Model":model_noTatil_fit,
          "All Including Model":model_all_fit}
#fitting the daily training data

In [22]:
def forecast_update(testData,model,tatilData):
    forecast = pd.DataFrame(np.zeros(testData.index.size),index=testData.index,columns=["Consumption"])
    for i,j in testData.iteritems() :
        forecast.loc[i]= model.predict(exogenous=np.array(tatilData.loc[i]).reshape(1,-1),n_periods=1)[0]
        model.update(testData.loc[i],exogenous=np.array(tatilData.loc[i]).reshape(1,-1))
    return forecast



In [23]:
type(daily_data) is pd.DataFrame

True

In [24]:
## EXOGENOUS VARIABLES FOR TEST

bayram_b_hs_df = pd.DataFrame(tatil_data[["Bayram","Haftasonu"]]["2019-01-01":"2019-12-31"])
bayram_b_hs_df.index = daily_data["2019"].index

bayram_b_t_df = pd.DataFrame(tatil_data[["Bayram","Tatil"]]["2019-01-01":"2019-12-31"])
bayram_b_t_df.index = daily_data["2019"].index

bayram_t_hs_df = pd.DataFrame(tatil_data[["Tatil","Haftasonu"]]["2019-01-01":"2019-12-31"])
bayram_t_hs_df.index = daily_data["2019"].index

bayram_b_df = pd.DataFrame(tatil_data["Bayram"]["2019-01-01":"2019-12-31"])
bayram_b_df.index = daily_data["2019"].index

bayram_t_df = pd.DataFrame(tatil_data["Tatil"]["2019-01-01":"2019-12-31"])
bayram_t_df.index = daily_data["2019"].index

bayram_hs_df = pd.DataFrame(tatil_data["Haftasonu"]["2019-01-01":"2019-12-31"])
bayram_hs_df.index = daily_data["2019"].index


In [25]:
#Forecasts with one step at a time and updating the model

forecast_onlyBayram = forecast_update(daily_data["Consumption"]["2019-01-01":"2019-12-31"],model_onlyBayram_fit,bayram_b_df)
forecast_onlyTatil = forecast_update(daily_data["Consumption"]["2019-01-01":"2019-12-31"],model_onlyTatil_fit,bayram_t_df)
forecast_onlyHaftasonu = forecast_update(daily_data["Consumption"]["2019-01-01":"2019-12-31"],model_onlyHaftasonu_fit,bayram_hs_df)


forecast_noBayram = forecast_update(daily_data["Consumption"]["2019-01-01":"2019-12-31"],model_noBayram_fit,bayram_t_hs_df)
forecast_noHaftasonu = forecast_update(daily_data["Consumption"]["2019-01-01":"2019-12-31"],model_noHaftasonu_fit,bayram_b_t_df)
forecast_noTatil = forecast_update(daily_data["Consumption"]["2019-01-01":"2019-12-31"],model_noTatil_fit,bayram_hs_b_df)

forecast_all = forecast_update(daily_data["Consumption"]["2019-01-01":"2019-12-31"],model_all_fit,tatil_data["2019-01-01":"2019-12-31"])
forecast_noExog = pd.DataFrame(np.zeros(daily_data["Consumption"]["2019-01-01":"2019-12-31"].index.size),index=daily_data["Consumption"]["2019-01-01":"2019-12-31"].index,columns=["Consumption"])

for i,j in daily_data["Consumption"]["2019-01-01":"2019-12-31"].iteritems() :
      forecast_noExog.loc[i]= model_noExog_fit.predict(n_periods=1)[0]
      model_noExog_fit.update(daily_data["Consumption"]["2019-01-01":"2019-12-31"].loc[i])

models_forecast = {"No Exogenous Model":forecast_noExog,
         "Only Bayram Exog. Model":forecast_onlyBayram,
         "Only Tatil Exog. Model":forecast_onlyTatil,
         "Only Haftasonu Exog. Model":forecast_onlyHaftasonu,
         "No Bayram Exog. Model":forecast_noBayram,                  
         "No Haftasonu Model":forecast_noHaftasonu,
        "No Tatil Model":forecast_noTatil,
         "All Including Model":forecast_all}

NameError: name 'bayram_hs_b_df' is not defined

In [ ]:
mapes = {}
for model in models_forecast :
    print("\n")
    print("-"*10,model,"-"*10)
    errors = (daily_data_test["2019-01-01":"2019-12-31"]-models_forecast[model])
    errors_percentage = ((errors/daily_data_test["2019-01-01":"2019-12-31"])*100)
    models_errors[model] = [errors,errors_percentage]
    MPE = errors_percentage.mean()[0]
    MAPE = abs(errors_percentage).mean()[0]
    mapes[model] = MAPE 

In [ ]:
print("Best Mape is: ",min(mapes, key=mapes.get))

In [ ]:
models[min(mapes, key=mapes.get)].summary()

# Saving the best models predicts

In [ ]:
index_column = pd.date_range(start ='2019-1-1', end = '2019-12-31', freq ='D')
best_pred = pd.DataFrame(models_forecast[min(mapes, key=mapes.get)],index=index_column)
best_pred.to_csv("the_best_arima_pred.csv",index = True)


In [ ]:
mapes_df = pd.DataFrame.from_dict(mapes, orient='index')
mapes_df.to_csv("mapes_arima.csv",index=True)

In [ ]:
mapes_df